In [3]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (10,6))
np.set_printoptions(precision =4, suppress=True)


In [4]:
# Import two datasets
colnames = ["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE","FRST_DTE", "OLTV", 
            "OCLTV", "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP","NUM_UNIT", "OCC_STAT", 
            "STATE", "ZIP_3", "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]
Data_A = pd.read_table('D:/1-Intern/Project_1/2018Q1/Acquisition_2018Q1.txt', 
                       sep = '|', header = None, names = colnames)
colnames_P = ["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
              "Adj.Month.To.Mat", "Maturity.Date", "MSA", "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
              "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS",
              "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"]
Data_P = pd.read_table('D:/1-Intern/Project_1/2018Q1/Performance_2018Q1.txt', 
                       sep = '|', header = None, names = colnames_P,low_memory=False)

In [5]:
df = pd.merge(Data_A, Data_P, on = 'LOAN_ID')

In [12]:
df.head

<bound method NDFrame.head of               LOAN_ID ORIG_CHN               Seller.Name  ORIG_RT  ORIG_AMT  \
0        100001040173        R        QUICKEN LOANS INC.     4.25    453000   
1        100001040173        R        QUICKEN LOANS INC.     4.25    453000   
2        100001040173        R        QUICKEN LOANS INC.     4.25    453000   
3        100001040173        R        QUICKEN LOANS INC.     4.25    453000   
4        100001040173        R        QUICKEN LOANS INC.     4.25    453000   
...               ...      ...                       ...      ...       ...   
9332045  999998783494        R  CALIBER HOME LOANS, INC.     4.25    358000   
9332046  999998783494        R  CALIBER HOME LOANS, INC.     4.25    358000   
9332047  999998783494        R  CALIBER HOME LOANS, INC.     4.25    358000   
9332048  999998783494        R  CALIBER HOME LOANS, INC.     4.25    358000   
9332049  999998783494        R  CALIBER HOME LOANS, INC.     4.25    358000   

         ORIG_TRM ORI

In [8]:
df['UPB_From_Last_Month']=df['LAST_UPB'].groupby(df['LOAN_ID']).shift(1)
df['UPB_From_Last_Month']

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
             ...    
9332045    349763.69
9332046    349239.33
9332047    348713.11
9332048    348185.03
9332049    347655.08
Name: UPB_From_Last_Month, Length: 9332050, dtype: float64

In [9]:
# Define a function to calculate origination monthly payment  
def Monthly_Payment_Calculation(x, y,z):
    return (x * (y / 1200)* ((1 + (y / 1200))**z)) / (((1+( y/1200))**z) -1)

df['Montly_Payment'] = round(Monthly_Payment_Calculation(df.loc[:, 'ORIG_AMT'], df.loc[:, 'ORIG_RT'], df.loc[:, 'ORIG_TRM']),2)
df['Montly_Payment']

0          2228.49
1          2228.49
2          2228.49
3          2228.49
4          2228.49
            ...   
9332045    1761.14
9332046    1761.14
9332047    1761.14
9332048    1761.14
9332049    1761.14
Name: Montly_Payment, Length: 9332050, dtype: float64

In [10]:
#Calculate monthly interest should be paid
df['Monthly_Interest_Payment']= df['UPB_From_Last_Month'] * (df['LAST_RT']/1200)
df['Monthly_Interest_Payment']

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
9332045    1238.746402
9332046    1236.889294
9332047    1235.025598
9332048    1233.155315
9332049    1231.278408
Name: Monthly_Interest_Payment, Length: 9332050, dtype: float64

In [11]:
#Calculate monthly principal should be paid
df['Scheduled_Monthly_Principal_Payment']= df['Montly_Payment']- df['Monthly_Interest_Payment']
df['Scheduled_Monthly_Principal_Payment']

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
              ...    
9332045    522.393598
9332046    524.250706
9332047    526.114402
9332048    527.984685
9332049    529.861592
Name: Scheduled_Monthly_Principal_Payment, Length: 9332050, dtype: float64

In [12]:
# Here is the part that how many principal of loan  has been deducted. 
df['Actual_Loan_Amount_Deducted'] = df['UPB_From_Last_Month']- df['LAST_UPB']
df['Actual_Loan_Amount_Deducted'] 

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
            ...  
9332045    524.36
9332046    526.22
9332047    528.08
9332048    529.95
9332049    531.83
Name: Actual_Loan_Amount_Deducted, Length: 9332050, dtype: float64

In [13]:
#creat a column to calculate the total loan payment made 
df['Actual_Monthly_Payment'] = df['Actual_Loan_Amount_Deducted']+ df['Monthly_Interest_Payment']
df['Actual_Monthly_Payment']

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
9332045    1763.106402
9332046    1763.109294
9332047    1763.105598
9332048    1763.105315
9332049    1763.108408
Name: Actual_Monthly_Payment, Length: 9332050, dtype: float64

In [14]:
def SMM(u,v,x,y):
    return(u - v)/(x - y)

df['SMM'] = round( SMM(df.loc[: , 'Actual_Monthly_Payment'], df.loc[: , 'Montly_Payment'],
                       df.loc[: , 'UPB_From_Last_Month'], df.loc[: , 'Scheduled_Monthly_Principal_Payment']),8)

In [15]:
#calculate CPR 
def CPR(x):
    return(1-(1-x)**12)

df['CPR']= round(CPR(df.loc[:, 'SMM']), 8) 
df['CPR']

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
9332045    0.000068
9332046    0.000068
9332047    0.000068
9332048    0.000068
9332049    0.000068
Name: CPR, Length: 9332050, dtype: float64

In [23]:
SMM_CPR_Each_Loan =  df[['SMM', 'CPR']].groupby(df['LOAN_ID'])
list(SMM_CPR_Each_Loan )

[(100001040173,
           SMM       CPR
  0        NaN       NaN
  1        NaN       NaN
  2        NaN       NaN
  3        NaN       NaN
  4        NaN       NaN
  5        NaN       NaN
  6        NaN       NaN
  7   0.000001  0.000013
  8   0.000001  0.000013
  9   0.000001  0.000013
  10  0.000001  0.000013
  11  0.000001  0.000013
  12  0.000001  0.000013
  13  0.000001  0.000013
  14  0.000001  0.000013
  15  0.000001  0.000013
  16  0.000001  0.000013
  17  0.000001  0.000013
  18  0.000001  0.000013
  19  0.000001  0.000013
  20  0.000001  0.000013
  21  0.000001  0.000013
  22  0.000001  0.000013),
 (100002370993,
           SMM       CPR
  23       NaN       NaN
  24       NaN       NaN
  25       NaN       NaN
  26       NaN       NaN
  27       NaN       NaN
  28       NaN       NaN
  29       NaN       NaN
  30  0.000148  0.001776
  31  0.000007  0.000090
  32  0.000008  0.000090
  33  0.000008  0.000090
  34  0.000073  0.000871
  35  0.001703  0.020248
  36  0.000120  

In [21]:
# for 'Zero.Bal.Code' equals 1, the loan of 'UPB_From_Last_Month' is the same as UPS in this month, so SMM and CPR should be calculated differently
for loan in df[df['Zero.Bal.Code']== 1.0]:
    df['SMM']= (df.loc[:,'UPB_From_Last_Month']- df.loc[: , 'Montly_Payment'])/(df.loc[: , 'UPB_From_Last_Month']- df.loc[: , 'Scheduled_Monthly_Principal_Payment'])
    df['CPR'] = round(1-(1-df.loc[:, 'SMM'])**12, 8)
